# Produccion electrica en base a la metereología

La idea principal del trabajo es ser capaces de predecir como va a afectar a que aumenten la produccion electrica de energias renovables las precipitaciones, el viento y las horas de sol.
Como las horas de sol y el viento son fenomenos cuya causa efecto es prácticamente inmediato vamos a centrarnos en la produccion de energia hidroelectrica.

En base a esto tenemos varias cuestiones que queremos despejar:
- Cuanto tarda en notarse en la generación eléctrica de fuentes de energía renovables los fenomenos metereológicos.
- Que cantidad de precipitaciones son necesarias para aumentar la generación de energia eléctica.
- Existe realmente una relación entre la metereologia y la producción electrica.


Como premisas partimos de :
- Vamos a considerar solo el poll de energia que proporciona Red Electrica de España (REE)
- Vamos a considerar que las empresas no trabajan bajo mala praxis y que intentan optimizar el uso de energias renovables.
- Se va a dividir el estudio por provincias, comunidades autonomas y a nivel estatal.

Como origenes de datos para el estudio vamos a utilizar los datos proporcionados por:
- Información de REE (https://www.ree.es/es/apidatos) obtenida mediante su API.
- Datos proporcionados por aemet(https://opendata.aemet.es/centrodedescargas/inicio), vamos a utilizar la libreria  aemet desarrollada por Pablo Moreno (https://pypi.org/project/python-aemet/).

Como requisitos para la ejecución del proyecto es necesario la instalación de la libreria python Aemet(pip install python-aemet) e instalar la libreria request



In [2]:
from aemet import Aemet,Estacion
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import requests
import time

# Leemos la clave de la API de AEMET desde un fichero ubicado en la misma ruta que este notebook
with open('API_KEY_AEMET','r') as file:
    API_KEY_AE=file.read()

    

In [3]:
# Obtenemos la lista de estaciones de medicion de aemet 
info_estaciones=Estacion.get_estaciones(API_KEY_AE)

# Creamos un objeto Aemet para usar los metodos de la libreria aemet
aemet=Aemet(API_KEY_AE)


In [4]:
# Lectura de datos de AEMET


# Definimos funciones que vamos a utilizar para leer los datos de aement
def estaciones_prov (prov,lista_estaciones):
    '''Dada una provincia y una lista de diccionarios de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.'''
    lista_id=[]
    for estacion in lista_estaciones:
        if estacion['provincia']==prov.upper():
            lista_id.append(estacion['indicativo'])
    return lista_id

help(estaciones_prov)


def lectura_dias(date_ini,date_end,estaciones):
    '''Dado un json del objeto Estacion, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    if date_ini[0:4]<'2016-01':
        date_ini="2016-01-01T00:00:00UTC"
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    for element in estaciones:
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element['indicativo'])
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(0.1) # para evitar errores por nº de lecturas.
    return valores_diarios;

help(lectura_dias)

# Pasamos los datos ha dataframes para su procesado
estaciones=pd.DataFrame(info_estaciones)

date_ini="2020-12-30T00:00:00UTC"
date_end="2020-12-31T00:00:00UTC"

wheather=pd.DataFrame(lectura_dias(date_ini,date_end,info_estaciones))

print(max(wheather['fecha']),min(wheather['fecha']))


#Rellenamos los NaN por 0 para precipitaciones. Y para temperatura los NaN por la termperatura del dia anterior


Help on function estaciones_prov in module __main__:

estaciones_prov(prov, lista_estaciones)
    Dada una provincia y una lista de diccionarios de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.

Help on function lectura_dias in module __main__:

lectura_dias(date_ini, date_end, estaciones)
    Dado un json del objeto Estacion, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.

2020-12-31 2020-12-20


In [16]:
wheather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 20 columns):
altitud        216 non-null object
dir            201 non-null object
fecha          216 non-null object
horaPresMax    115 non-null object
horaPresMin    115 non-null object
horaracha      201 non-null object
horatmax       201 non-null object
horatmin       200 non-null object
indicativo     216 non-null object
nombre         216 non-null object
prec           214 non-null object
presMax        115 non-null object
presMin        115 non-null object
provincia      216 non-null object
racha          201 non-null object
sol            112 non-null object
tmax           202 non-null object
tmed           202 non-null object
tmin           202 non-null object
velmedia       204 non-null object
dtypes: object(20)
memory usage: 33.8+ KB


In [36]:
# Leemos las regiones de ree obtenidas desde (https://www.ree.es/es/apidatos) desde un fichero ubicado en la misma ruta que este notebook
region_ree=pd.read_csv('REGION_REE',header=0,index_col='Region')
region_ccaa=region_ree[region_ree.geo_limit=='ccaa']

region_ccaa.iloc[1]

geo_limit    ccaa
geo_id          5
Name: Aragón, dtype: object

In [41]:
# Obtenemos los datos de REE a traves de su API.


parametros={'start_date':'2019-01-01T00:00','end_date':'2019-01-01T10:00','time_trunc':'day','geo_trunc':'electric_system','geo_limit':'ccaa','geo_ids':'5'}
URL_BAL='https://apidatos.ree.es/es/datos/balance/balance-electrico'
URL_GEN='https://apidatos.ree.es/es/datos/generacion/estructura-generacion'
url_prueba='https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2014-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids=7'
~ree_bal=requests.get(URL_BAL,params=parametros)
#ree_gen=requests.get(URL_GEN,params=parametros)
#if ree_gen.raise_for_status()!=None:
 #   print('Error en lectura')
    
#if ree_bal.raise_for_status()!=None:
  #  print('Error en lectura')
#ree=pd.DataFrame(request)

# Paso el resultado a str
#type(request.data.decode())

     
# Paso el str a json para poder pasarlo a Dataframe
#prueba=json.loads(request.data)


In [42]:
print(ree_bal.url)
print(ree_bal.status_code)
print(ree_bal.elapsed)
print(ree_bal.encoding)
print(ree_bal.status_code)
print(ree_bal.ok)
print(ree_bal.history)

print(ree_gen.url)
print(ree_gen.status_code)
print(ree_gen.elapsed)
print(ree_gen.encoding)
print(ree_gen.status_code)
print(ree_gen.ok)
print(ree_gen.history)

https://apidatos.ree.es/es/datos/balance/balance-electrico?start_date=2019-01-01T00%3A00&end_date=2019-01-01T10%3A00&time_trunc=day&geo_trunc=electric_system&geo_limit=ccaa&geo_ids=5
400
0:00:00.277196
None
400
False
[]
https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2019-01-01T00%3A00&end_date=2019-01-01T10%3A00&time_trunc=day&geo_trunc=electric_system&geo_limit=ccaa&geo_ids=5
500
0:00:00.324712
UTF-8
500
False
[]


In [133]:
# sacar los ficheros a json()

{'data': {'type': 'Generación por tecnología',
  'id': 'gen1',
  'attributes': {'title': 'Generación por tecnología',
   'last-update': '2020-01-10T08:15:46.000+01:00',
   'description': None},
  'meta': {'cache-control': {'cache': 'MISS'}}},
 'included': [{'type': 'Hidráulica',
   'id': '10288',
   'groupId': '1',
   'attributes': {'title': 'Hidráulica',
    'description': None,
    'color': '#0090d1',
    'type': 'Renovable',
    'magnitude': None,
    'composite': False,
    'last-update': '2020-01-10T08:15:21.000+01:00',
    'values': [{'value': 19307.962,
      'percentage': 0.08370628760227758,
      'datetime': '2019-01-01T00:00:00.000+01:00'}]}},
  {'type': 'Turbinación bombeo',
   'id': '1445',
   'groupId': '1',
   'attributes': {'title': 'Turbinación bombeo',
    'description': None,
    'color': '#007cf9',
    'type': 'No-Renovable',
    'magnitude': None,
    'composite': False,
    'last-update': '2019-06-12T17:49:29.000+02:00',
    'values': [{'value': 761.737,
      'pe

In [85]:
renovable=data ['included'][0]
no_renovable=data ['included'][1]
demanda=data ['included'][2]

In [91]:
renovable['attributes']



{'title': 'Renovable',
 'last-update': '2020-01-10T08:15:21.000+01:00',
 'description': None,
 'magnitude': None,
 'content': [{'type': 'Hidráulica',
   'id': '10288',
   'groupId': 'Renovable',
   'attributes': {'title': 'Hidráulica',
    'description': None,
    'color': '#0090d1',
    'type': 'distinct',
    'magnitude': None,
    'composite': False,
    'last-update': '2020-01-10T08:15:21.000+01:00',
    'values': [{'value': 19307.962,
      'percentage': 0.29485581120822324,
      'datetime': '2019-01-01T00:00:00.000+01:00'}],
    'total': 19307.962}},
  {'type': 'Eólica',
   'id': '10291',
   'groupId': 'Renovable',
   'attributes': {'title': 'Eólica',
    'description': None,
    'color': '#6fb114',
    'type': 'distinct',
    'magnitude': None,
    'composite': False,
    'last-update': '2020-01-10T08:15:42.000+01:00',
    'values': [{'value': 37895.955,
      'percentage': 0.57871682951496,
      'datetime': '2019-01-01T00:00:00.000+01:00'}],
    'total': 37895.955}},
  {'type

In [95]:
drenov=pd.DataFrame.from_dict(renovable['attributes'])
drenov

,title,last-update,description,magnitude,content
0,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Hidráulica', 'id': '10288', 'groupId..."
1,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Eólica', 'id': '10291', 'groupId': '..."
2,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Solar fotovoltaica', 'id': '1458', '..."
3,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Solar térmica', 'id': '1459', 'group..."
4,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Hidroeólica', 'id': '1455', 'groupId..."
5,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Otras renovables', 'id': '10292', 'g..."
6,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Residuos renovables', 'id': '10295',..."
7,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Generación renovable', 'id': 'Renova..."
